In [25]:
!pip install -q pymupdf sentence-transformers faiss-cpu transformers torch accelerate pandas

In [26]:
import pandas as pd
import fitz
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [27]:
# Configuración inicial
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EMBEDDING_MODEL = "all-MiniLM-L6-v2"
LLM_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

In [28]:
# Carga modelos
modelo_emb = SentenceTransformer(EMBEDDING_MODEL).to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)
modelo = AutoModelForCausalLM.from_pretrained(LLM_MODEL, torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32).to(DEVICE)

In [29]:
# Variables globales
index = None
df_segmentos = None

In [30]:
def extraer_segmentos(pdf_path):
    doc = fitz.open(pdf_path)
    segmentos = []
    for pagina in doc:
        texto = pagina.get_text("text")
        for parrafo in texto.split('\n'):
            limpio = ' '.join(parrafo.strip().split())
            if len(limpio.split()) >= 15:
                max_words = 150
                words = limpio.split()
                for i in range(0, len(words), max_words):
                    segmento = ' '.join(words[i:i+max_words])
                    segmentos.append(segmento)
    return segmentos

In [31]:
def construir_indice(segmentos):
    embeddings = modelo_emb.encode(segmentos, normalize_embeddings=True)
    index = faiss.IndexFlatIP(embeddings.shape[1])
    index.add(embeddings)
    return index, pd.DataFrame({"texto": segmentos})

In [32]:
def recuperar_contexto(pregunta, k=3, umbral=0.7):
    if index is None or df_segmentos is None:
        return None
    emb_pregunta = modelo_emb.encode([pregunta], normalize_embeddings=True)
    distancias, indices = index.search(emb_pregunta, k)
    contextos_relevantes = []
    for i, dist in zip(indices[0], distancias[0]):
        if dist >= umbral:
            contextos_relevantes.append(df_segmentos.iloc[i]["texto"])
    return "\n\n".join(contextos_relevantes) if contextos_relevantes else None

In [33]:
def generar_respuesta(pregunta, contexto):
    if not contexto:
        return "No tengo información suficiente en el documento para responder a tu pregunta."

    system_prompt = """Responde la pregunta de manera precisa y concisa basado únicamente en este contexto:
    {contexto}

    Pregunta: {pregunta}
    Respuesta:""".format(contexto=contexto, pregunta=pregunta)

    inputs = tokenizer(system_prompt, return_tensors="pt").to(DEVICE)
    outputs = modelo.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.1
    )
    respuesta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return respuesta.split("Respuesta:")[-1].strip()

In [34]:
def evaluar_respuesta(respuesta_obtenida, respuesta_esperada, tipo_pregunta):
    if tipo_pregunta == "fuera_dominio":
        # Para preguntas fuera de dominio, lo correcto es no responder con información del PDF
        if "no tengo información" in respuesta_obtenida.lower():
            return 5
        else:
            return 1
    else:
        # Para preguntas dentro del dominio
        respuesta_obtenida_clean = respuesta_obtenida.lower().strip()
        respuesta_esperada_clean = respuesta_esperada.lower().strip()

        # Coincidencia exacta o contiene toda la respuesta esperada
        if (respuesta_obtenida_clean == respuesta_esperada_clean or
            respuesta_esperada_clean in respuesta_obtenida_clean):
            return 5

        # Coincidencia parcial (algunos elementos clave presentes)
        elementos_clave = [word for word in respuesta_esperada_clean.split() if len(word) > 3]

        # Manejo del caso cuando no hay elementos clave
        if not elementos_clave:
            # Si no hay palabras clave, verificamos si hay alguna coincidencia básica
            palabras_comunes = set(respuesta_obtenida_clean.split()) & set(respuesta_esperada_clean.split())
            return 3 if palabras_comunes else 1

        coincidencias = sum(1 for word in elementos_clave if word in respuesta_obtenida_clean)
        proporcion_coincidencias = coincidencias / len(elementos_clave)

        if proporcion_coincidencias >= 0.75:
            return 4
        elif proporcion_coincidencias >= 0.5:
            return 3
        elif proporcion_coincidencias >= 0.25:
            return 2
        else:
            return 1

In [35]:
def ejecutar_pruebas(pdf_path, csv_preguntas_path, csv_resultados_path):
    # 1. Procesar PDF
    print("Procesando PDF...")
    segmentos = extraer_segmentos(pdf_path)
    if not segmentos:
        raise ValueError("No se pudo extraer contenido útil del PDF")

    global index, df_segmentos
    index, df_segmentos = construir_indice(segmentos)
    print(f"PDF procesado: {len(segmentos)} segmentos extraídos")

    # 2. Cargar preguntas de prueba
    df_preguntas = pd.read_csv(csv_preguntas_path)

    # 3. Ejecutar pruebas
    resultados = []
    for _, fila in df_preguntas.iterrows():
        pregunta = fila['pregunta']
        respuesta_esperada = fila['respuesta_esperada']
        tipo_pregunta = fila['tipo_pregunta']

        if tipo_pregunta == "fuera_dominio":
            respuesta_obtenida = generar_respuesta(pregunta, None)  # Forzamos sin contexto
        else:
            contexto = recuperar_contexto(pregunta)
            respuesta_obtenida = generar_respuesta(pregunta, contexto)

        # Evaluación más sofisticada
        precision = evaluar_respuesta(respuesta_obtenida, respuesta_esperada, tipo_pregunta)

        resultados.append({
            'tipo_pregunta': tipo_pregunta,
            'pregunta': pregunta,
            'respuesta_esperada': respuesta_esperada,
            'respuesta_obtenida': respuesta_obtenida,
            'precision': precision,
            'observaciones': ""
        })

    # 4. Guardar resultados
    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_csv(csv_resultados_path, index=False)
    print(f"Pruebas completadas. Resultados guardados en {csv_resultados_path}")

    # Resumen estadístico
    print("\nResumen de resultados:")
    total_preguntas = len(resultados)

    for tipo in ['simple', 'compleja', 'fuera_dominio']:
        subset = df_resultados[df_resultados['tipo_pregunta'] == tipo]
        count = len(subset)

        if count > 0:
            print(f"\nPreguntas {tipo} ({count}):")
            print(f" - Perfectas (5): {sum(subset['precision'] == 5)} ({sum(subset['precision'] == 5)/count:.1%})")
            print(f" - Buenas (4): {sum(subset['precision'] == 4)} ({sum(subset['precision'] == 4)/count:.1%})")
            print(f" - Aceptables (3): {sum(subset['precision'] == 3)} ({sum(subset['precision'] == 3)/count:.1%})")
            print(f" - Deficientes (2): {sum(subset['precision'] == 2)} ({sum(subset['precision'] == 2)/count:.1%})")
            print(f" - Incorrectas (1): {sum(subset['precision'] == 1)} ({sum(subset['precision'] == 1)/count:.1%})")
            print(f" - Puntaje promedio: {subset['precision'].mean():.2f}/5")

In [36]:
# Ejecutar pruebas
pdf_path = "/content/drive/MyDrive/Proyectos/ChatBot/Redes_entrada.pdf"
csv_preguntas_path = "/content/drive/MyDrive/Proyectos/ChatBot/preguntas_prueba.csv"
csv_resultados_path = "/content/drive/MyDrive/Proyectos/ChatBot/resultados_prueba.csv"

try:
    ejecutar_pruebas(pdf_path, csv_preguntas_path, csv_resultados_path)
except Exception as e:
    print(f"Error durante las pruebas: {str(e)}")

Procesando PDF...
PDF procesado: 447 segmentos extraídos


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Pruebas completadas. Resultados guardados en /content/drive/MyDrive/Proyectos/ChatBot/resultados_prueba.csv

Resumen de resultados:

Preguntas simple (45):
 - Perfectas (5): 4 (8.9%)
 - Buenas (4): 1 (2.2%)
 - Aceptables (3): 4 (8.9%)
 - Deficientes (2): 2 (4.4%)
 - Incorrectas (1): 34 (75.6%)
 - Puntaje promedio: 1.64/5

Preguntas compleja (41):
 - Perfectas (5): 0 (0.0%)
 - Buenas (4): 0 (0.0%)
 - Aceptables (3): 1 (2.4%)
 - Deficientes (2): 9 (22.0%)
 - Incorrectas (1): 31 (75.6%)
 - Puntaje promedio: 1.27/5

Preguntas fuera_dominio (16):
 - Perfectas (5): 16 (100.0%)
 - Buenas (4): 0 (0.0%)
 - Aceptables (3): 0 (0.0%)
 - Deficientes (2): 0 (0.0%)
 - Incorrectas (1): 0 (0.0%)
 - Puntaje promedio: 5.00/5
